# Итоговая структура базы данных
## Таблица customers

Сущность: клиент.

Ключи и связи

customer_id – PRIMARY KEY

используется как внешний ключ в transactions.customer_id

Поля

customer_id – INTEGER NOT NULL — уникальный идентификатор клиента

first_name – VARCHAR(50) NOT NULL — имя

last_name – VARCHAR(50) — фамилия (может отсутствовать)

gender – VARCHAR(10) NOT NULL — пол (F, M, Female, Male, U и т.п.)

dob – DATE — дата рождения (не у всех заполнена)

job_title – TEXT — должность (может быть пустой)

job_industry_category – VARCHAR(50) NOT NULL — отрасль

wealth_segment – VARCHAR(50) NOT NULL — сегмент богатства

deceased_indicator – CHAR(1) NOT NULL — признак смерти (Y / N)

owns_car – VARCHAR(5) NOT NULL — владеет ли автомобилем (Yes / No)

address – TEXT NOT NULL — адрес

postcode – INTEGER NOT NULL — почтовый индекс

state – VARCHAR(50) NOT NULL — штат

country – VARCHAR(50) NOT NULL — страна (в данных всегда Australia, но поле оставлено общим)

property_valuation – INTEGER NOT NULL — оценка недвижимости

## Таблица products

Сущность: товар (велосипед).

Ключи и связи

product_id – PRIMARY KEY

используется как внешний ключ в transactions.product_id

Поля

product_id – INTEGER NOT NULL — уникальный идентификатор товара

brand – VARCHAR(50) — бренд (может быть пустым)

product_line – VARCHAR(50) — линейка товара (Road, Standard, Touring, …)

product_class – VARCHAR(50) — класс (low, medium, high)

product_size – VARCHAR(50) — размер (small, medium, large)

standard_cost – NUMERIC(10,2) — себестоимость (может быть пустой)

list_price – NUMERIC(10,2) NOT NULL — цена продажи

## Таблица transactions

Сущность: отдельная покупка (транзакция).

Ключи и связи

transaction_id – PRIMARY KEY

customer_id – FOREIGN KEY → customers(customer_id)

product_id – FOREIGN KEY → products(product_id)

Поля

transaction_id – INTEGER NOT NULL — уникальный идентификатор транзакции

customer_id – INTEGER NOT NULL — ссылка на клиента

product_id – INTEGER NOT NULL — ссылка на товар

transaction_date – DATE NOT NULL — дата покупки

online_order – BOOLEAN — онлайн-заказ или покупка офлайн (TRUE / FALSE / NULL если неизвестно)

order_status – VARCHAR(50) NOT NULL — статус (Approved или Cancelled

# Описание нормализации (до 3НФ)

Исходно в файле у нас по сути были две логические сущности, смешанные через общие идентификаторы:
клиенты и транзакции, плюс информация о товарах.

## 1НФ

Все исходные данные уже находятся в табличном виде: каждая ячейка содержит одно значение, строки однородны по структуре.

На уровне транзакций можно выбрать ключ transaction_id — он однозначно идентифицирует запись.

Т.е. требования 1НФ выполняются.

## 2НФ

Если рассматривать «большую» логическую таблицу «Транзакции + данные клиента + данные товара», то:

неключевые атрибуты клиента (имя, адрес, пол и т.д.) зависят только от customer_id, а не от всей строки транзакции;

неключевые атрибуты товара (бренд, линия, класс…) зависят только от product_id.

Чтобы убрать частичные зависимости, мы:

Выделили отдельную таблицу customers, в которой все атрибуты зависят только от customer_id.

Выделили отдельную таблицу products, в которой атрибуты зависят только от product_id.

В таблице transactions оставили только ссылки (customer_id, product_id) и собственные поля транзакции.

Так мы устранили зависимости «части составного ключа» и перевели модель в 2НФ.

## 3НФ

Далее проверяем транзитивные зависимости:

В customers все поля напрямую зависят от customer_id. Например, state и postcode не используются как ключи для других полей внутри таблицы — они просто характеристики клиента.

В products все поля зависят от product_id.

В transactions все неключевые поля (transaction_date, online_order, order_status) зависят только от transaction_id, через него же определяются customer_id и product_id.

Следовательно, каждый неключевой атрибут зависит от ключа, от всего ключа и только от ключа — это и есть 3НФ.

In [2]:
# 

In [1]:
===== Таблица клиентов ====
DROP TABLE IF EXISTS transactions;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS customers;

CREATE TABLE customers (
    customer_id         INTEGER         PRIMARY KEY,
    first_name          VARCHAR(50)     NOT NULL,
    last_name           VARCHAR(50),
    gender              VARCHAR(10)     NOT NULL,
    dob                 DATE,
    job_title           TEXT,
    job_industry_category VARCHAR(50)   NOT NULL,
    wealth_segment      VARCHAR(50)     NOT NULL,
    deceased_indicator  CHAR(1)         NOT NULL,
    owns_car            VARCHAR(5)      NOT NULL,
    address             TEXT            NOT NULL,
    postcode            INTEGER         NOT NULL,
    state               VARCHAR(50)     NOT NULL,
    country             VARCHAR(50)     NOT NULL,
    property_valuation  INTEGER         NOT NULL
);

==== Таблица товаров ====
CREATE TABLE products (
    product_id      INTEGER         PRIMARY KEY,
    brand           VARCHAR(50),
    product_line    VARCHAR(50),
    product_class   VARCHAR(50),
    product_size    VARCHAR(50),
    standard_cost   NUMERIC(10,2),
    list_price      NUMERIC(10,2)   NOT NULL
);

==== Таблица транзакций ====
CREATE TABLE transactions (
    transaction_id  INTEGER         PRIMARY KEY,
    customer_id     INTEGER         NOT NULL,
    product_id      INTEGER         NOT NULL,
    transaction_date DATE           NOT NULL,
    online_order    BOOLEAN,
    order_status    VARCHAR(50)     NOT NULL
);

==== Внешние ключи ====
ALTER TABLE transactions
    ADD CONSTRAINT fk_tr_customer
    FOREIGN KEY (customer_id)
    REFERENCES customers(customer_id);

ALTER TABLE transactions
    ADD CONSTRAINT fk_tr_product
    FOREIGN KEY (product_id)
    REFERENCES products(product_id);


SyntaxError: invalid syntax (2753433724.py, line 1)